In [13]:
import pandas as pd
import numpy as np
import os
import os.path, time
import zipfile as zp
import re
import random

from scipy import optimize
import matplotlib.pyplot as plt
import matplotlib.axes as ax
from scipy import stats
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel
from statsmodels.base.model import LikelihoodModel
from sklearn import preprocessing
import statsmodels.formula.api as sm
from numpy.random import randn
from numpy.random import seed
from scipy.stats import pearsonr
import unicodedata
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [14]:
from scipy.stats import pearsonr
def calculate_pvalues(df):
    df = df.dropna()._get_numeric_data()
    dfcols = pd.DataFrame(columns=df.columns)
    pvalues = dfcols.transpose().join(dfcols, how='outer')
    for r in df.columns:
        for c in df.columns:
            pvalues[r][c] = round(pearsonr(df[r], df[c])[1], 4)
    return pvalues

In [15]:
def subID_list():
    subID = ['797','806','809','826','843','845','855','856','857','1031','1153','1154','1157','1174','1262','1264','1266',
        '1364','1367','1379','1380','1381','1384','1435','1465','1467','1468','1489','1492','1493','1494','1495','1498','1519',
        '1520','1527','1532','1536','1541','1721','1722','1737','1741','1788','1791','1810','1812','1833','1934',
            '2310','2316','2318','2323','2338','2339','2340','2348','2353','2356','2357','2358','2377','2378',
             '2381','2383','2385','2386','2387','2388','2389','2390','2391','2394','2395','2396','2399',
             '2401','2402','2404','2405','2407','2434','2444','2456',
            '2433','2447','2477','2479','2489','2547','2548','2549','2553','2555','2558','2559','2578','2951','2958',
            '2961','2962','2963','2964','2967','2968','2975','2977','2978','2981','2983',
             '2998','2999','3051','3052','3053']
    return subID

### Combined all variables

In [16]:
iv_df = pd.read_csv("/Users/zyy219/Documents/Risk_project/Data/Independent_variable_1.csv")
dv_df = pd.read_csv("/Users/zyy219/Documents/Risk_project/Data/Dependent_variable.csv")
ratio_rt_df = pd.read_csv("/Users/zyy219/Documents/Risk_project/Data/ratio_rt_df.csv")
age_gender_df = pd.read_csv("/Users/zyy219/Documents/Risk_project/Data/age_gender_df.csv")

df = iv_df.drop(['Unnamed: 0', 'subID','E_RP','F_RP','S_RP','R_RP','S_RP.1','E_EB','F_EB','S_EB','R_EB','S_EB.1'], axis = 1)

DV = dv_df[['subID','alpha_EU','gamma_EU','alpha_ESVT','M_ESVT','gamma_ESVT']]
DV.loc[:,'RT'] = ratio_rt_df['RT']
DV.loc[:,'risk_Ratio'] = ratio_rt_df['risk_ratio']

df.loc[:,'alpha_EU']= DV['alpha_EU']
df.loc[:,'gamma_EU'] = DV['gamma_EU']
df.loc[:,'alpha_ESVT'] = DV['alpha_ESVT']
df.loc[:,'M_ESVT'] = DV['M_ESVT']
df.loc[:,'gamma_ESVT']  = DV['gamma_ESVT']
df.loc[:,'RT'] = DV['RT']
df.loc[:,'risk_Ratio'] = DV['risk_Ratio']
df.loc[:,'gender'] = age_gender_df['Gender']
df.loc[:,'age'] = age_gender_df['Age']

//anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
//anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [17]:
df

,E_LI,F_LI,S_LI,R_LI,S_LI.1,IUS,lot_pos,lot_neg,drm_pos,drm_neg,...,crt,alpha_EU,gamma_EU,alpha_ESVT,M_ESVT,gamma_ESVT,RT,risk_Ratio,gender,age
0,14.0,15.0,14.0,10.0,24.0,62.0,9.0,12.0,3.063750,1.823125,...,1.0,0.624,3.520,1.000,25.087,59.220,106.3125,0.38000,0,26
1,10.0,13.0,15.0,15.0,13.0,69.0,7.0,9.0,3.916250,1.135000,...,3.0,0.609,1.892,1.000,22.349,28.261,124.3750,0.40625,0,27
2,23.0,16.0,23.0,17.0,36.0,96.0,9.0,11.0,4.333125,1.406250,...,3.0,0.698,1.552,1.000,38.166,33.913,101.3750,0.42500,0,25
3,18.0,20.0,22.0,26.0,33.0,73.0,10.0,11.0,2.687500,2.093125,...,2.0,0.727,1.532,1.000,38.399,47.973,89.7500,0.49000,1,27
4,17.0,23.0,16.0,24.0,26.0,55.0,14.0,6.0,4.896875,0.291875,...,0.0,0.600,0.793,1.000,20.661,9.222,132.8125,0.42250,0,33
5,18.0,17.0,23.0,12.0,30.0,81.0,12.0,7.0,5.250000,0.426875,...,1.0,0.560,2.149,1.000,16.955,20.506,195.8125,0.33000,0,38
6,21.0,13.0,32.0,19.0,38.0,108.0,10.0,11.0,3.291250,0.729375,...,1.0,0.290,2.095,3.977,4.685,7.391,245.3750,0.23875,1,51
7,20.0,17.0,11.0,12.0,27.0,67.0,14.0,6.0,4.623750,0.000000,...,0.0,0.000,1.185,1.000,0.000,1.185,395.8125,0.36875,0,55
8,19.0,26.0,25.0,21.0,34.0,79.0,12.0,12.0,3.665625,2.188125,...,1.0,1.462,0.020,123.350,9.027,3.385,98.3125,0.75250,1,26
9,34.0,40.0,23.0,38.0,34.0,72.0,11.0,12.0,3.355000,1.780000,...,0.0,0.000,0.000,24.123,65.127,2.804,72.5625,0.50625,0,35


### Performed PCA on all subjects

In [18]:
x= df.dropna().to_numpy()
# Standardizing the features
x = StandardScaler().fit_transform(x)
pca = PCA(n_components=4)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['PC1', 'PC2','PC3','PC4'])
pca.explained_variance_ratio_


array([0.21493017, 0.1303461 , 0.08286957, 0.07678706])

### loading: weights of linear combination
PCA loadings are the coefficients of the linear combination of the original variables from which the principal components (PCs) are constructed.

In [19]:
feature_imp = pca.components_
loading_matrix_linear_com = pd.DataFrame(feature_imp.T, columns=['PC1', 'PC2', 'PC3','PC4'])
loading_matrix_linear_com
#pd.DataFrame(np.round(feature_imp.T,2)).to_csv("c2_df_imp.csv")

,PC1,PC2,PC3,PC4
0,0.129865,0.194927,-0.024231,0.342754
1,0.023481,0.294288,0.108525,0.233712
2,0.120008,0.261832,0.091489,0.277388
3,0.083112,0.265142,0.044924,0.228856
4,-0.004675,0.157017,-0.082087,0.417842
5,0.270587,-0.095408,-0.020393,0.015793
6,-0.170355,0.296339,-0.092032,0.023380
7,0.271910,-0.043711,-0.103490,-0.134671
8,-0.205875,0.276894,-0.052531,-0.063470
9,0.316249,0.041879,-0.027009,0.037899


In [20]:
np.round(principalDf.cov(),3)

,PC1,PC2,PC3,PC4
PC1,6.507,0.000,0.000,0.000
PC2,0.000,3.946,0.000,-0.000
PC3,0.000,0.000,2.509,-0.000
PC4,0.000,-0.000,-0.000,2.325


### weekly 

In [21]:
risk_ratio_weekly = pd.read_csv("/Users/zyy219/Documents/Risk_project/Data/Weekly_measurement/risk_ratio_weekly.csv")

risk_EU_alpha_weekly  = pd.read_csv("/Users/zyy219/Documents/Risk_project/Data/Weekly_measurement/risk_EU_alpha_weekly.csv")
risk_EU_gamma_weekly  = pd.read_csv("/Users/zyy219/Documents/Risk_project/Data/Weekly_measurement/risk_EU_gamma_weekly.csv")
risk_ESVT_alpha_weekly  = pd.read_csv("/Users/zyy219/Documents/Risk_project/Data/Weekly_measurement/risk_ESVT_alpha_weekly.csv")
risk_ESVT_M_weekly  = pd.read_csv("/Users/zyy219/Documents/Risk_project/Data/Weekly_measurement/risk_ESVT_M_weekly.csv")
risk_ESVT_gamma_weekly  = pd.read_csv("/Users/zyy219/Documents/Risk_project/Data/Weekly_measurement/risk_ESVT_gamma_weekly.csv")


In [23]:
subID = subID_list()
week = [1,2,3,4,5,6,7,8]
pc3_score = np.zeros((len(week),len(subID)))
for i in range(len(week)):
    df['risk_Ratio'] = risk_ratio_weekly.iloc[:,i+1]
    df['alpha_EU'] = risk_EU_alpha_weekly.iloc[:,i+1]
    df['gamma_EU'] = risk_EU_gamma_weekly.iloc[:,i+1]
    df['alpha_ESVT'] = risk_ESVT_alpha_weekly.iloc[:,i+1]
    df['M_ESVT'] = risk_ESVT_M_weekly.iloc[:,i+1]
    df['gamma_ESVT'] = risk_ESVT_gamma_weekly.iloc[:,i+1]
    
    #print(i)
    pc3 = loading_matrix_linear_com['PC3']
    e = np.array(df).dot(np.array(pc3))
    pc3_score[i,:] = e


In [24]:
pc3_score_df = pd.DataFrame(data=pc3_score.T,columns = ['week1','week2','week3','week4','week5','week6','week7','week8'])
pc3_score_df
#pc3_score_df.to_csv("/Users/zyy219/Documents/Risk_project/Data/Weekly_measurement/pc3_score.csv")

,week1,week2,week3,week4,week5,week6,week7,week8
0,23.131758,24.538507,15.351041,66.922321,57.781996,303.254831,17.190506,303.271143
1,49.710616,529.563977,13.271433,13.872389,23.559375,25.629638,31.290550,28.958245
2,6.462700,-1.834662,26.555906,19.619269,24.520887,41.409224,45.350512,373.830857
3,13.532019,4.560259,41.980900,118293.556881,55.718241,76.172379,13.382750,662.011425
4,78.645813,-10.661788,-2.823647,-6.625650,-4.995082,623.810175,12.168802,16.674410
5,13.861529,51.564878,18.115157,259.802786,-6.865724,-3.803676,-11.968473,-4.971892
6,-4.374711,-22.478322,-25.557656,-23.264195,-25.246252,-24.653539,-29.269425,-2.319838
7,10063.315030,-26.579329,-32.011291,-27.964741,-23.423314,-31.939312,-25.934906,-25.825961
8,10643.777142,-4.873863,-0.295533,1.252513,-11.626137,-8.192498,-5.741234,-5.414906
9,-25.269167,-16.409075,-18.326893,-13.911565,-19.783919,0.265711,-11.854501,-10.555647


In [122]:
subID = subID_list()
R = ['855','856','857','1031','1364','1381','1435','1492','1493',
     '1519','1721','2338','2340','2356','2378','2387','2444','2553','2447']
N = [i for i in range(len(subID)) if subID[i] in R]


In [48]:
alpha_EU_h_df = df[df['alpha_EU'] >= np.median(df['alpha_EU'])] 
alpha_EU_l_df = df[df['alpha_EU'] < np.median(df['alpha_EU'])] 

risk_ratio_h_df = df[df['risk_Ratio'] >= np.median(df['risk_Ratio'])] 
risk_ratio_l_df = df[df['risk_Ratio'] < np.median(df['risk_Ratio'])] 

M_h_df = df[df['M_ESVT'] >= np.median(df['M_ESVT'])] 
M_l_df = df[df['M_ESVT'] < np.median(df['M_ESVT'])] 

alpha_ESVT_h_df = df[df['alpha_ESVT'] >= np.median(df['alpha_ESVT'])] 
alpha_ESVT_l_df = df[df['alpha_ESVT'] < np.median(df['alpha_ESVT'])] 

In [29]:
x= alpha_EU_h_df.dropna().to_numpy()
# Standardizing the features
x = StandardScaler().fit_transform(x)
pca = PCA(n_components=4)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['PC1','PC2','PC3','PC4'])
print(pca.explained_variance_ratio_)
feature_imp = pca.components_
pd.DataFrame(np.round(feature_imp.T,2)).to_csv("high_EU_alpha_cp.csv")

[0.21943348 0.16421494 0.09118722 0.07040862]


### loading correlations between the original variable and the principal components.


In [8]:
loadings = pca.components_.T * np.sqrt(pca.explained_variance_)
loading_matrix = pd.DataFrame(loadings, columns=['PC1', 'PC2', 'PC3','PC4'])